<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch02_DataFrames_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
from pyspark_helpers import *
sc, spark, conf = initspark()



### ***LAB:*** Use the regions and territories RDDs from the previous lab and convert them into DataFrames with meaningful schemas.


In [ ]:
regions = sc.textFile('hdfs://localhost:9000/regions')
regions = regions.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1]))
regionsdf = regions.toDF('RegionID:int, RegionName:string')
regionsdf.show()

In [ ]:
territories = sc.textFile('hdfs://localhost:9000/territories')
territories = territories.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1], int(x[2])))
territoriesdf = territories.toDF('TerritoryID:int, TerritoryName:string, RegionID: int')
territoriesdf.show()

### ***LAB:*** Using the df3 DataFrame, answer the following questions:

*   How many Platinum card purchases were there with a discount above $100?
*   Find the ten biggest discount amounts earned by women and show just the purchase amount, discount, and date.



In [ ]:
filename = f'{datapath}/finance/CreditCard.csv'
cc = sc.textFile(filename)
first = cc.first()
cc = cc.filter(lambda x : x != first)
import datetime
cc = cc.map(lambda x : x.split(',')) 
cc = cc.map(lambda x : (x[0][1:], x[1][1:-1], datetime.datetime.strptime(x[2], '%d-%b-%y').date(), x[3], x[4], x[5], float(x[6])))
df = spark.createDataFrame(cc)
df = cc.toDF('City: string, Country: string, Date: date, CardType: string, TranType: string, Gender: string, Amount: double')
df2 = df.withColumn('Discount', df.Amount * .03)
df3 = df2.drop(df2.Country)


print(df3.where("CardType = 'Platinum' and Discount > 100").count())
display(df3.where("Gender = 'F'").orderBy('Amount', ascending = False).select('Amount', 'Discount', 'Date'))

### ***LAB:*** Read the Products file from the JSON folder and categories from the CSVHeaders folder, then join them displaying just the product and category IDs and names, and sort by categoryID then productID. 

**Hint:** Drop the ambiguous column after the join.

In [ ]:
products = spark.read.json(f'{datapath}/northwind/JSON/products')
#products.show()
products.printSchema()

categories = spark.read.csv(f'{datapath}/northwind/CSVHeaders/categories', header = True, inferSchema = True)
#categories.show()
categories.printSchema()

c = categories
p = products
c.join(p, c.CategoryID == p.categoryid).drop(p.categoryid).select('CategoryID', 'CategoryName', 'productid', 'productname').orderBy('categoryid', 'productid').show()
